In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/uditrawat/Desktop/CardioX'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.mlpro.constants import *
from src.mlpro.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import os
import requests
import zipfile
from src.mlpro import logger
from src.mlpro.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                response = requests.get(self.config.source_URL)
                response.raise_for_status()  

                with open(self.config.local_data_file, 'wb') as file:
                    file.write(response.content)

                logger.info(
                    f"File downloaded to {self.config.local_data_file}")
                logger.info(f"Response headers: {response.headers}")
            except requests.exceptions.RequestException as e:
                logger.error(f"Error downloading file: {str(e)}")
                raise
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-02 21:33:02,547: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-10-02 21:33:02,550: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-02 21:33:02,552: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-02 21:33:02,553: INFO: common: created directory at: artifacts]
[2024-10-02 21:33:02,553: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-02 21:33:04,694: INFO: 548635367: File downloaded to artifacts/data_ingestion/data.zip]
[2024-10-02 21:33:04,695: INFO: 548635367: Response headers: {'Connection': 'keep-alive', 'Content-Length': '3021753', 'Cache-Control': 'max-age=300', 'Content-Security-Policy': "default-src 'none'; style-src 'unsafe-inline'; sandbox", 'Content-Type': 'application/zip', 'ETag': 'W/"9609b72b7ca1d0a12f7147bd77da4e84abe1727d7169fbba17d6af5d0b6a9aeb"', 'Strict-Transport-Security': 'max-age=31536000', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'deny', 'X-XSS-Protection':